## 1. Знакомство с проектом и данными

#### Цель: 
"Для каждого client_id в наборе тестов ('test.csv') вы должны предсказать вероятность для default переменной. Итоговый файл должен содержать в заголовоке: 
* client_id - идентификатор заемщика;
* default - вероятность дефолта по кредиту."

Исходные данные представленные в виде обучающей выборки: train(73,799 записей) и тестовой: test(36,349 записей). Данные выборок хорошо сбалансированны по всем параметрам (пропорции классов в обоих файлах по всем параметрам сохраняются), но искомые классы в парамтре default не сбалансированны и представленны в пропорции: 12,7% / 87,3%

##### Учитывая, что искомые классы не сбалансированны, а так же то, что нас интересует максималый процент верных предсказаний по обеим классам - в качестве ключевой метрики для оценки точности будем использовать F-меру (ну и тем более по ней оценивается рейтинг в лидерборде!).

In [ ]:
# импортируем библиотеки

import numpy as np
import pandas as pd
import seaborn as sns
import sklearn

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_selection import f_classif, mutual_info_classif
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold, StratifiedShuffleSplit
from sklearn.model_selection import cross_val_score, cross_validate, GridSearchCV
from sklearn.metrics import mean_squared_error, f1_score, accuracy_score
from sklearn.metrics import balanced_accuracy_score, precision_recall_curve
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, roc_auc_score
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler, LabelEncoder, OneHotEncoder
from itertools import combinations, combinations_with_replacement

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
# подгрузим дополнительные настройки

import warnings
warnings.simplefilter('ignore')

# import matplotlib.pyplot as plt2
from matplotlib import pylab as plt
%matplotlib inline

pd.set_option('display.width', 140, 'display.max_columns', None)

In [ ]:
# зафиксируем версию пакетов, чтобы эксперименты были воспроизводимы:
!pip freeze > requirements.txt
SEED = 100

In [ ]:
# функция обработки бинарных категориальных признаков (в бинарный числовой):
def binary_futures(*dfs, **columns_args):
    for df in dfs:
        for col, arg in columns_args.items():
            df[col] = (df[col] == arg).astype('int32')

#### Посмотрим на обучающие и тестовые данные:

In [ ]:
DATA_DIR = '/kaggle/input/sf-scoring/'
train = pd.read_csv(DATA_DIR+'train.csv')
test = pd.read_csv(DATA_DIR+'/test.csv')

In [ ]:
# Как видим в данных присутсвует 6 категориальных признаков, 12 числовых и один целевой параметр (в обучающей выборке): default - флаг дефолта по кредиту.
train.info()

In [ ]:
print(train.shape)
train.sample(5)

In [ ]:
print(test.shape)
test.sample(5)

In [ ]:
# Посмотрим какой процент дефолтных записей:
train.default.value_counts(normalize=True)*100

##### Видим, что выборка не сбалансированна и вероятно потребуется добавить записей по дефолтным клиентам.

In [ ]:
# Проверим наличие пропусков в данных:
print('Пропуски в train:', train.isnull().sum()[train.isnull().sum() > 0])
print('Пропуски в test:', test.isnull().sum()[test.isnull().sum() > 0])

In [ ]:
# Можно построить быстрый анализ данный с помощью profiling, но не станем и проведем данный анализ самостоятельно.
# import pandas_profiling
# pandas_profiling.ProfileReport(train)

## 2. Анализ и обработка бинарных, категориальных и ординальных признаков:

#### 'sex'

In [ ]:
# Посмотрим на пропорции выборки:
print(train.sex.value_counts())
pd.crosstab(train.sex, train.default, normalize='index')*100

##### Выраженной зависимости дефолта по привязке к полу - не наблюдается (видимо можем прогнозировать, что данный признак не значительный).

In [ ]:
# Проверим пропорции выборки в тестовом наборе данных:
print(test.sex.value_counts())

In [ ]:
# Преобразуем в числовой-бинарный, где: 1 == 'M', 0 == 'F'
train.sex = (train.sex == 'M').astype('int32')
test.sex = (test.sex == 'M').astype('int32')

# binary_futures(train, test, {'sex': 'M'})

#### 'car', 'car_type'

In [ ]:
# Посмотрим на пропорции выборки:
print(train.car.value_counts())
print(train.car_type.value_counts())
pd.crosstab([ train.car, train.car_type], train.default, normalize='index')*100

Видим, что приблизительно треть выборки имеют автомобиль, иномаркой владеют около 20% выборки.
При этом вероятность дефолта существенно снижается только для категории лиц владеющих иномаркой - 8,7% (для владельцев отечественного авто и не имеющих авто вероятность дефолта практически одинакова - 13,3% и 13,7% соответственно).

##### Следовательно делаем вывод, что признак "car" - не значительный.

In [ ]:
# Проверим пропорции выборки в тестовом наборе данных:
print(test.car.value_counts(normalize=True)*100)
print(test.car_type.value_counts(normalize=True)*100)

In [ ]:
# Преобразуем в числовой-бинарный, где: 1 == 'Y', 0 == 'N'
train.car = (train.car == 'Y').astype('int32')
test.car = (test.car == 'Y').astype('int32')

train.car_type = (train.car_type == 'Y').astype('int32')
test.car_type = (test.car_type == 'Y').astype('int32')

# binary_futures(train, test, {'car': 'Y, 'car_type': 'Y'})

#### 'foreign_passport'

In [ ]:
# Посмотрим на пропорции выборки:
print(train.foreign_passport.value_counts(normalize=True)*100)
pd.crosstab(train.foreign_passport, train.default, normalize='index')*100

##### Как видим для категории лиц имеющих загранпаспорт вероятность дефолта практически в два раза ниже, чем для остальных: 7,4% против 13,6%

In [ ]:
# Проверим пропорции выборки в тестовом наборе данных:
print(test.foreign_passport.value_counts(normalize=True)*100)

In [ ]:
# Преобразуем в числовой-бинарный, где: 1 == 'Y', 0 == 'N'
train.foreign_passport = (train.foreign_passport == 'Y').astype('int32')
test.foreign_passport = (test.foreign_passport == 'Y').astype('int32')

# binary_futures(train, test, {'foreign_passport': 'Y'})

#### 'education'
* SCH - High School (9-12 класс)
* UGR - Undergraduate (бакалавриат, 4 года)
* GRD - Graduate (магистратура, 6 лет)
* PGR - Postgraduate (докторантура, 10 лет)
* ACD - ... видимо что-то типа ученой степени

In [ ]:
# Проверим пропорции выборки в обучающем и тестовом наборе данных:
print(train.education.value_counts(normalize=True)*100)
print(test.education.value_counts(normalize=True)*100)

In [ ]:
# посмотрим на корреляцию уровня образования с дефолтом:
pd.crosstab(train.education, train.default, normalize='index')*100

In [ ]:
# проверим какова вероятность дефолта для клиентов с неуказанным уровнем образования:
na_edu_default = round(train[train.education.isna()].default.value_counts(normalize=True)*100, 2)
na_edu_default

##### Как видим вероятность дефолта для категории лиц с пропуском данных в образовании - 11,4%, в то время как для наиболее распространенной категории (SCH) - 15%. Следовательно заменять пропущенные данные самым распространенным - плохая идея. Заменим степень образования и пропуски на соответствующую вероятность дефолта.

In [ ]:
# создадим словарь для ординального преобразования, где в качестве величины будет соответтвующая вероятность дефолта:
edu_dict = round(pd.crosstab(train.education, train.default, normalize='index').iloc[:, 1]*100, 2)

# заменим значения в исследуемом параметре:
train.education = train.education.map(edu_dict)
test.education = test.education.map(edu_dict)

# для пропушенных данных по образованию заменим на соответствующую вероятность 11.4%:
train.education.fillna(na_edu_default[1], inplace=True)
test.education.fillna(na_edu_default[1], inplace=True)

# в результате получаем в качестве значения данного парамтера числовую вероятность дефолта по соотвествующей группе:
pd.crosstab(train.education, train.default)

#### 'app_date'

In [ ]:
# приведем значения к формату datetime:
train.app_date = train.app_date.apply(pd.to_datetime)
test.app_date = test.app_date.apply(pd.to_datetime)

In [ ]:
# добавим новый параметр - month:
train['month'] = train.app_date.dt.month
test['month'] = test.app_date.dt.month
pd.crosstab([train.month], train.default, normalize='index')*100

In [ ]:
# добавим новый параметр - clients_day, кол-во клиентов в день:
dates_dict = train.app_date.value_counts().to_dict()
train['clients_day'] = train.app_date.map(dates_dict)
test['clients_day'] = train.app_date.map(dates_dict)
train.clients_day.hist()

In [ ]:
# добавим новый параметр - numb_weeks, номер недели (на основе данных train):
train['numb_weeks'] = train.app_date.dt.week
test['numb_weeks'] = test.app_date.dt.week
sns.displot(train, x="numb_weeks", hue="default", multiple="fill")  # , kind="kde"

In [ ]:
# поиск зависимости вероятности дефолта от дня месяца, дня недели или декады результата не дал:
# train['day'] = train.app_date.dt.day
# train['day_of_week'] = train.app_date.dt.dayofweek
# train['decade'] = 0
# train['decade'][train.day <= 10] = 1
# train['decade'][train.day.between(11, 20)] = 2
# train['decade'][train.day > 20] = 3

In [ ]:
# теперь можем удалять исходный параметр:
train.drop(['app_date'], axis=1, inplace=True)
test.drop(['app_date'], axis=1, inplace=True)

### 'good_work'

In [ ]:
# Посмотрим на пропорции выборки:
print(train.good_work.value_counts(normalize=True)*100)
pd.crosstab(train.good_work, train.default, normalize='index')*100

##### Присутствует зависимость вероятности дефолта от наличия хорошей работы.

In [ ]:
# Проверим пропорции на тестовом наборе:
test.good_work.value_counts(normalize=True)*100

### 'region_rating'

In [ ]:
# Посмотрим на распределение:
pd.crosstab(train.region_rating, train.default, normalize='index')*100

##### Прослеживается хорошая линейная зависимость.

In [ ]:
# посмотрим на пропорции в обучающем и тестовом наборе данных:
print(train.region_rating.value_counts(normalize=True).sort_index()*100)
test.region_rating.value_counts(normalize=True).sort_index()*100

### 'work_address'

In [ ]:
# Посмотрим на распределение вероятности дефолта:
pd.crosstab(train.work_address, train.default, normalize='index')*100

In [ ]:
# Проверим пропорции выборки для обучащей и тестовых выборок:
print(train.work_address.value_counts(normalize=True).sort_index()*100)
test.work_address.value_counts(normalize=True).sort_index()*100

### 'home_address'

In [ ]:
# Посмотрим на пропорции вероятности дефолта:
pd.crosstab(train.home_address, train.default, normalize='index')*100

##### Наблюдается не линейное распределение (с пиком для home_address=2).

In [ ]:
# Проверим распределение для обучащей и тестовых выборок:
print(train.home_address.value_counts(normalize=True).sort_index()*100)
test.home_address.value_counts(normalize=True).sort_index()*100

In [ ]:
# учитывая вероятность дефолта для класса home_address=3 (11,6%), логичней поместить его на промежуточный (между 1 и 2):
train.home_address[train.home_address == 3] = 1.5
test.home_address[test.home_address == 3] = 1.5

# # учитывая малое кол-во выборки для класса "3" и низкую вероятность дефолта для этого же класса (11,6%), что ближе к классу "1", можем их объеденить:
# train.home_address[train.home_address == 3] = 1
# test.home_address[test.home_address == 3] = 1

pd.crosstab(train.home_address, train.default, normalize='index')*100

### 'sna'

In [ ]:
# Посмотрим на распределение вероятноти дефолта:
pd.crosstab(train.sna, train.default, normalize='index')*100

##### Видим, что для групп sna=3 и sna=2 близкие по значению вероятности дефолта.

In [ ]:
# посмотрим на пропорции выборки по категориям:
print(train.sna.value_counts(normalize=True).sort_index()*100)
test.sna.value_counts(normalize=True).sort_index()*100

In [ ]:
# # учитывая небольшое кол-во для группы sna=3 и близкое значение вероятности дефолта с группой sna=2 - можем их объеденить
# train.sna = train.sna.apply(lambda x: 2.5 if x in [2, 3] else x)
# test.sna = test.sna.apply(lambda x: 2.5 if x in [2, 3] else x)
# pd.crosstab(train.sna, train.default, normalize='index')*100

### 'first_time'

In [ ]:
# Посмотрим на распределение вероятноти дефолта:
pd.crosstab(train.first_time, train.default, normalize='index')*100

In [ ]:
# посмотрим на пропорции выборки по категориям:
print(train.first_time.value_counts(normalize=True).sort_index()*100)
test.first_time.value_counts(normalize=True).sort_index()*100

### Итоги анализа ("вероятности дефолта" обозначим как: "ВД"):
* sex - пол заемщика, - не формирует выраженной зависимости ВД;
* car - флаг наличия автомобиля, - рассмотрев совместно с признаком car_type можно утвердать что данный признак не формирует зависимости к ВД;
* car_type - флаг автомобиля иномарки, - формирует хорошо выраженную зависимость ВД;
* foreign_passport - наличие загранпаспорта, - присутствует выраженная зависимость ВД;
* education - уровень образования, - присутствуют пропуски данных; преобразовали категории в соответствущие вероятности дефолта.
* app_date - дата подачи заявки, - на основании сформированно несколько новых признаков с выраженной зависимостью ВД;
* good_work - флаг наличия “хорошей” работы, - присутствует выраженная зависимость ВД;
* region_rating - рейтинг региона, - присутствует выраженная зависимость ВД;
* home_address - категоризатор домашнего адреса, - присутствует выраженная зависимость ВД;
* work_address - категоризатор рабочего адреса, - присутствует выраженная зависимость ВД;
* sna - связь заемщика с клиентами банка - можно отнести к ординальному типу признаков с линейным ростом ВД;
* first_time - давность наличия информации о заемщике - обратная линейная зависимость ВД.

#### 'sex' + 'car' = 'sex_car' - заменим два бесполезных признака на один полезный

In [ ]:
pd.crosstab([train.sex, train.car], train.default, normalize='index')*100

In [ ]:
train['sex_car'] = 0
train['sex_car'][(train.car == 1)&(train.sex == 0)] = 1
train['sex_car'][(train.car == 1)&(train.sex == 1)] = 2
train['sex_car'][(train.car == 0)&(train.sex == 0)] = 3
train['sex_car'][(train.car == 0)&(train.sex == 1)] = 4

test['sex_car'] = 0
test['sex_car'][(test.car == 1)&(test.sex == 0)] = 1
test['sex_car'][(test.car == 1)&(test.sex == 1)] = 2
test['sex_car'][(test.car == 0)&(test.sex == 0)] = 3
test['sex_car'][(test.car == 0)&(test.sex == 1)] = 4

pd.crosstab(train.sex_car, train.default, normalize='index')*100

In [ ]:
# Удалим ранее отбракованные (или оптимизированные в новые признаки) данные:
train.drop(['sex', 'car'], axis=1, inplace=True)  # 
test.drop(['sex', 'car'], axis=1, inplace=True)   # 

## 3. Анализ и обработка числовых признаков:

### 'client_id'

In [ ]:
# Убедимся что все записи уникальны (train + test):
id_all = train.client_id.append(test.client_id)
id_all.shape[0] == id_all.nunique()

In [ ]:
# заметив что у client_id почти 100% корреляция с параметром month, проверим его очередность:
train.sort_values('client_id')

##### Делаем вывод, что данный параметр соответствует порядковому номеру подачи заявки клиентом (начиная с 01.01.2014 и заканчивая 30.04.2014). Данный параметр не несет ни какой фундаментальной (логической) ценности для выявления закономерности по вероятности дефолта.

### 'age'

In [ ]:
# Посмотрим на возрастное распределение по дефолтам:
sns.displot(train, x="age", col="default", bins=52)

In [ ]:
# Для адекватного восприятия данного распределения (дефолтов по возрасту) - приведем расчеты к относительному виду, (%):
df_age = pd.DataFrame()
df_age['default_0'] = train.age[train.default == 0].value_counts()
df_age['default_1'] = train.age[train.default == 1].value_counts()
df_age['default_%'] = df_age['default_1'] / (df_age['default_0'] + df_age['default_1']) * 100
df_age['age'] = df_age.index

# видим параболическую зависимость вероятности дефолта от возраста (c минимумом в области 55 лет и возрастанием по краям диапазона):
df_age.plot(x='age', y='default_%', kind='scatter')

In [ ]:
# для лучшей апроксимации сгладим статистику объеденив в группы по 5-летним диапазонам:
df_age.age = df_age.age.apply(lambda x: (x // 5) * 5)
df_age_group5 = df_age.groupby(['age'])[['default_0', 'default_1']].sum()
df_age_group5['%%'] = df_age_group5['default_1'] / (df_age_group5['default_0'] + df_age_group5['default_1'])
df_age_group5['age'] = df_age_group5.index
df_age_group5.plot(x='age', y='%%', kind='scatter')

In [ ]:
# используем данные вероятности дефолта для возрастных групп в обучающем и тестовом наборах данных:
dict_default_by_age = round(df_age_group5['%%']*100, 3).to_dict()

# проведем преобразование:
train['age'] = train.age.apply(lambda x: (x // 5) * 5).map(dict_default_by_age)
test['age'] = test.age.apply(lambda x: (x // 5) * 5).map(dict_default_by_age)

### 'decline_app_cnt'

In [ ]:
# Посмотрим какое имеем колличественное распределение:
train.decline_app_cnt.value_counts()

In [ ]:
# сгруппируем малочисленные группы и посмотрим на вероятности дефолтов:
train.decline_app_cnt[train.decline_app_cnt > 3] = 3
test.decline_app_cnt[test.decline_app_cnt > 3] = 3
pd.crosstab(train.decline_app_cnt, train.default, normalize='index')*100

In [ ]:
# # другие варианты группировок и dummy-разложения не принесли улучшения значения для f1:
# train = pd.get_dummies(train, columns=['decline_app_cnt'])
# test = pd.get_dummies(test, columns=['decline_app_cnt'])

# # создадим словарь вероятности дефолта для соответствующих группировок (по кол-ву предыдущих отказов):
# dict_default_by_decline_app_cnt = pd.crosstab(train.decline_app_cnt, train.default, normalize='index')*100

# # добавим параметр характеризующий вероятность дефолта для соответствующей группы "количества отказанных прошлых заявок":
# train['decline_app_cnt'] = train.decline_app_cnt .map(dict_default_by_decline_app_cnt.iloc[:, -1])
# test['decline_app_cnt'] = test.decline_app_cn.map(dict_default_by_decline_app_cnt.iloc[:, -1])
# train.decline_app_cnt.value_counts(normalize=True)*100

### 'score_bki'

In [ ]:
# Посмотрим на распределение:
sns.distplot(train.score_bki)

In [ ]:
sns.displot(train, x="score_bki", hue="default", multiple="fill")  # , kind="kde"

##### Видим, что на малочисленных хвостах ((-4;-3) и (-0.5;0.2))  присутствуют разброд и шатания.
##### Но попытка произвести обработку для сглаживания распределения не привела к улучшению точности модели.

In [ ]:
# # сгрупируем левый хвост:
# train.score_bki[train.score_bki < - 3.5] = - 3.5
# test.score_bki[test.score_bki < - 3.5] = - 3.5

# # произведем сглаживание данных с шагом 0.2:
# train.score_bki = train.score_bki.apply(lambda x: (x*10//2)/5)
# test.score_bki = test.score_bki.apply(lambda x: (x*10//2)/5)
# # произведем сглаживание данных с шагом 0.1:
# train.score_bki = train.score_bki.apply(lambda x: x*10//1/10)
# test.score_bki = test.score_bki.apply(lambda x: x*10//1/10)

# # предположим что значения больше 0 - это ошибка и исправим для них знак:
# train.score_bki[train.score_bki > 0] = - train.score_bki
# test.score_bki[test.score_bki > 0] = - test.score_bki

# sns.displot(train, x="score_bki", hue="default", multiple="fill")

### 'bki_request_cnt'

In [ ]:
# Посмотрим на распределение:
pd.crosstab(train.bki_request_cnt, train.default)

In [ ]:
# Проведем группировку кол-ва запросов для балансировки и сглаживания зависимости:
train.bki_request_cnt[train.bki_request_cnt > 16] = 16
train.bki_request_cnt[train.bki_request_cnt.between(11, 15)] = 11
train.bki_request_cnt[train.bki_request_cnt.between(8, 10)] = 8
train.bki_request_cnt[train.bki_request_cnt.between(4, 7)] = 4

test.bki_request_cnt[test.bki_request_cnt > 16] = 16
test.bki_request_cnt[test.bki_request_cnt.between(11, 15)] = 11
test.bki_request_cnt[test.bki_request_cnt.between(8, 10)] = 8
test.bki_request_cnt[test.bki_request_cnt.between(4, 7)] = 4

pd.crosstab(train.bki_request_cnt, train.default, normalize='index')*100

### 'income'

In [ ]:
# Посмотрим на распределение доходов:
sns.distplot(train.income, bins=25)

In [ ]:
# Без логарифмического преобразования тут не обойтись:
train['income'] = np.log(train['income'] + 1)
test['income'] = np.log(test['income'] + 1)

sns.distplot(train.income, bins=100)

In [ ]:
# произведем сглаживание данных с шагом 0.5:
train.income = train.income.apply(lambda x: ((x * 10) // 5) / 2)
test.income = test.income.apply(lambda x: ((x * 10) // 5) / 2)

# Проведем группировки для сглаживания зависимости:
train.income[train.income < 8.5] = 8.5
test.income[test.income < 8.5] = 8.5
train.income[train.income > 12] = 12
test.income[test.income > 12] = 12

pd.crosstab(train.income, train.default, normalize='index')*100

In [ ]:
# # Попытка развернуть параболу к линейному виду, не привела к улучшению ф-меры на тесте:
# train.income = abs(train.income - 9.5)
# test.income = abs(test.income - 9.5)
# pd.crosstab(train.income, train.default, normalize='index')*100

### Итоги анализа ("вероятности дефолта" обозначим как: "ВД"):
* client_id - идентификатор клиента - не имеет смысла, просто порядковый номер клиента;
* age - возраст заемщика - формирует параболическую зависимость ВД с минимумом в районе 55 лет и резким ростом после 70 лет;
* decline_app_cnt - количество отказанных прошлых заявок - хорошо выраженная зависимость ВД, наблюдается практический линейный рост;
* score_bki - скоринговый балл по данным из БКИ - имеет структуру нормального распределения, формирует параболическую зависимость ВД с минимумов в районе -2,8 и бОльшим максимумом в правом хвосте;
* bki_request_cnt - количество запросов в БКИ - повторяет зависимость как и у параметра decline_app_cnt, - практический линейный рост;
* income - доход заемщика - после логарифмического преобразования имеет структуру нормального распределения, но при этом зависимсоть ВД носит не линейный характер;

In [ ]:
# Удалим ранее отбракованные (или оптимизированные в новые признаки) данные:
train.drop(['client_id'], axis=1, inplace=True)  # 
test.drop(['client_id'], axis=1, inplace=True)   # 

## 4. Матрица корреляции и оценка значимости:

In [ ]:
plt.figure(figsize=(16, 8))
sns.heatmap(train.corr().abs(), annot=True, cmap='coolwarm', fmt='.3f', annot_kws={'size':10})

In [ ]:
# Для оценки значимости переменных будем использовать функцию mutual_info_classif:
imp_cat = pd.Series(mutual_info_classif(train.drop(columns='default', axis=1), train['default'], 
                                        discrete_features='auto', random_state=100), 
                    index=train.drop(columns='default', axis=1).columns.values)
imp_cat.sort_values()

In [ ]:
# Учитывая высокую корреляцию между 'month' и 'numb_weeks', и отсутствие "значимости" для 'numb_weeks', - удалим последний:
train.drop(['numb_weeks'], axis=1, inplace=True)   
test.drop(['numb_weeks'], axis=1, inplace=True)

## 5. Feature Engineering:

##### Синтетическая генерация новых признаков (из имеющихся) не увенчалась положительной прибавкой для f1.

In [ ]:
# # Разобъем оставшиеся признаки на три категории:
# columns_bins = ['car_type', 'good_work', 'home_address', 'work_address', 'sna', 'first_time', 'foreign_passport', 'month']
# columns_cat = ['education', 'age', 'decline_app_cnt', 'region_rating', 'income']
# columns_numb = ['score_bki', 'clients_day', 'defaults_day', 'numb_weeks', 'bki_request_cnt']

# # На основании этих категорий сформируем комбинации:
# bins_3 = list(combinations(columns_bins, 3))
# cat_3 = list(combinations(columns_cat, 3))
# numb_3 = list(combinations(columns_numb, 3))

# print('combinations(columns_bins, 3): ', len(bins_3))
# print('combinations(columns_cat, 3): ', len(cat_3))
# print('combinations(columns_numb, 3): ', len(numb_3))

In [ ]:
# # 1. Добавим весь перечень синтезированных парметров bins_3:
# for x, y, z in bins_3:
#     train['dif1_' + str(x) + str(y) + str(z)] = train[x] + train[y] - train[z]
#     train['dif2_' + str(x) + str(y) + str(z)] = train[x] - train[y] + train[z]
#     train['dif3_' + str(x) + str(y) + str(z)] = train[x] - train[y] - train[z]

# # 2. С помощью mutual_info_classif отберем лучшие из новых параметров:
# dif1_home_address sna foreign_passport    0.016630
# dif2_car_type home_address first_time     0.015406
# dif3_car_type work_address sna            0.015467

# 3. Добавим новые параметры в train и test:
# train['1'] = train['home_address'] + train['sna'] - train['foreign_passport']
# train['2'] = train['car_type'] - train['home_address'] + train['first_time']
# train['3'] = train['car_type'] - train['work_address'] - train['sna']
# test['1'] = test['home_address'] + test['sna'] - test['foreign_passport']
# test['2'] = test['car_type'] - test['home_address'] + test['first_time']
# test['3'] = test['car_type'] - test['work_address'] - test['sna']

In [ ]:
# # 1. Добавим весь перечень синтезированных парметров cat_3:
# for x, y, z in cat_3:
#     train['dif1_' + str(x) + str(y) + str(z)] = train[x] + train[y] - train[z]
#     train['dif2_' + str(x) + str(y) + str(z)] = train[x] - train[y] + train[z]
#     train['dif3_' + str(x) + str(y) + str(z)] = train[x] - train[y] - train[z]
#     train['mult1_' + str(x) + str(y) + str(z)] = (train[x] - train[y]) / (train[x] + train[y])
#     train['mult2_' + str(x) + str(y) + str(z)] = (train[x] - train[z]) / (train[x] + train[z])
#     train['mult3_' + str(x) + str(y) + str(z)] = (train[y] - train[z]) / (train[y] + train[z])
    
    
# # 2. С помощью mutual_info_classif отберем лучшие из новых параметров:
# dif3_education decline_app_cnt region_rating     0.015614

# # 3. Добавим новые параметры в train и test:
# train['4'] = train['decline_app_cnt'] - train['bki_request_cnt'] - train['education']
# train['5'] = train['decline_app_cnt'] * train['education']
# test['4'] = test['decline_app_cnt'] - test['bki_request_cnt'] - test['education']
# test['5'] = test['decline_app_cnt'] * test['education']

In [ ]:
# # 1. Добавим весь перечень синтезированных парметров numb_3:
# for x, y, z in numb_3:
#     train['dif1_' + str(x) + str(y) + str(z)] = train[x] + train[y] - train[z]
#     train['dif2_' + str(x) + str(y) + str(z)] = train[x] - train[y] + train[z]
#     train['dif3_' + str(x) + str(y) + str(z)] = train[x] - train[y] - train[z]
#     train['mult1_' + str(x) + str(y) + str(z)] = pow(abs((train[x] - train[y]) * train[z]), 0.5)
#     train['mult2_' + str(x) + str(y) + str(z)] = pow(abs((train[x] - train[z]) * train[y]), 0.5)
#     train['mult3_' + str(x) + str(y) + str(z)] = pow(abs((train[y] - train[z]) * train[x]), 0.5)

# # 2. С помощью mutual_info_classif отберем лучшие из новых параметров:
# mult1_score_bki clients_day numb_weeks         0.016457
# mult_score_bki income                      0.018898
# sum_score_bki region_rating                0.022869

# # 3. Добавим новые параметры в train и test:
# train['6'] = train['score_bki'] + train['clients_day']
# train['7'] = train['score_bki'] * train['income']
# train['8'] = train['score_bki'] + train['region_rating']
# test['6'] = test['score_bki'] + test['clients_day']
# test['7'] = test['score_bki'] * test['income']
# test['8'] = test['score_bki'] + test['region_rating']

In [ ]:
# imp_cat = pd.Series(mutual_info_classif(train.drop(columns='default', axis=1), train['default'], 
#                                         discrete_features='auto', random_state=100), 
#                     index=train.drop(columns='default', axis=1).columns.values)
# imp_cat.sort_values()

## 6. Нормализация и балансировка обучающей выборки:

In [ ]:
# Проведем нормализацию обработанных параметров:
scaler=RobustScaler()    # на test показал слегка лучше результат, чем MinMaxScaler()
columns = train.drop(columns='default', axis=1).columns.values
train[columns] = scaler.fit_transform(train.drop(columns='default', axis=1).values)
test[columns] = scaler.transform(test.values)

# Для балансировки (класса default=1) разделим нашу выборку train на обучающую и валидационну):
X = train.drop(columns='default', axis=1)
y = train['default'].values
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.20, random_state=SEED)

# Протестируем два варианта балансировки:
# 1. RandomOverSampler расширения выборки:
from imblearn.over_sampling import RandomOverSampler
oversam = RandomOverSampler(sampling_strategy='minority', random_state=SEED)
X_over, y_over = oversam.fit_resample(X_train, y_train)

# 2. SMOTE расширения выборки:
from imblearn.over_sampling import SMOTE
smot = SMOTE(sampling_strategy='minority', random_state=SEED)
X_smot, y_smot = smot.fit_resample(X_train, y_train)

X_train.shape, X_over.shape, X_smot.shape

## 7. Подбор гиперпараметров:

In [ ]:
C = np.logspace(0, -2, 10)       # [1, 1e-1, 1e-2, 1e-3]
iters = [15, 25, 50, 75]
epsilon_stop = [1e-4]
class_weight = [None]         # выбрать в случаи подбора параметров для сбалансированных выборок
# class_weight = ['balanced']   # для несбалансированной выборки
param_grid = [
    {'C': C,
     'penalty': ['l1'],
     'solver': ['liblinear'],
     'class_weight': class_weight,
     'multi_class': ['auto'],
     'max_iter': iters,
     'tol': epsilon_stop},
    {'C': C,
     'penalty': ['l2'],
     'solver': ['liblinear', 'newton-cg', 'lbfgs', 'saga'],
     'class_weight': class_weight,
     'multi_class': ['auto'],
     'max_iter': iters,
     'tol':epsilon_stop},
    {'C': C,
     'penalty': ['elasticnet'],
     'solver': ['saga'],
     'class_weight': class_weight,
     'multi_class': ['auto'],
     'max_iter': iters,
     'tol':epsilon_stop},
    {'C': ['none'],
     'penalty': ['none'],
     'solver': ['lbfgs', 'sag'],
     'class_weight': class_weight,
     'multi_class': ['auto'],
     'max_iter': iters,
     'tol':epsilon_stop}]

##### Далее код скрыт, так как его выполнение занимает много времени, а параметры уже зафиксированны.

In [ ]:
# Разбиение на обучающую и валидационную часть, используя один из двух вариантов расширения:
# X_train, y_train = X_over, y_over
# X_train, y_train = X_smot, y_smot

# model = LogisticRegression()  # solver='liblinear'
# model.fit(X_train, y_train)

# # Произведем поиск гиперпараметров GridSearchCV при помощи перебора параметров по сетке param_grid:
# grid_search = GridSearchCV(model, param_grid, scoring='f1', n_jobs=-1, cv=5)
# grid_search.fit(X_train, y_train)
# # grid_search.best_params_

# # Печатаем параметры развернуто:
# best_model = grid_search.best_estimator_
# best_parameters = best_model.get_params()
# for param_name in sorted(best_parameters.keys()):
#     print('\t%s: %r' % (param_name, best_parameters[param_name]))

### Выбор одного из вариантов обучающей выборки и соотвествующих подобранных гиперпараметров:

In [ ]:
# 1. Изначальная обучающая выборка train с соответствующими гиперпараметрами:
# X, y = X_train, y_train
# model = LogisticRegression(C=1, class_weight='balanced', max_iter=75, penalty='none', solver='sag', tol=1e-4, multi_class='auto')

# 2. Расширение обучающего датасэта с помощью RandomOverSampler и соответствующие оптипальные гиперпараметры под него:
X, y = X_over, y_over
model = LogisticRegression(C=1e-1, class_weight=None, max_iter=50, penalty='l2', solver='liblinear', tol=1e-4, multi_class='auto')
# model = LogisticRegression(C=1e-2, class_weight=None, max_iter=25, penalty='l2', solver='lbfgs', tol=1e-4, multi_class='auto')

# 3. Расширение обучающего датасэта с помощью SMOTE (и соответствующие оптипальные гиперпараметры под него):
# X, y = X_smot, y_smot
# model = LogisticRegression(C=1e-1, class_weight=None, max_iter=100, penalty='l2', solver='newton-cg', tol=1e-4, multi_class='auto')

## 8. Обучение модели:

### Проверка на наличие паразитных параметров:

##### Проверив на test видим, что исключение любого из оставшихся параметров ухудшает результат расчета модели по f1.

In [ ]:
# # Для начала зафиксируем результат по метрике f1 (с порогом отсечения 0,5):
# model_check = model
# model_check.fit(X, y)
# y_pred = model_check.predict(X_valid)
# f1_score(y_valid, y_pred).T

In [ ]:
# # Определившись с выбором гиперпараметров и вариантом обучающей выборки еще раз проверим, исключая каждый параметры по очереди, что паразитных не осталось:
# dict_exc_param = {}
# for param in X.columns.values:
#     model_check_1 = model
#     model_check_1.fit(X.drop(columns=param, axis=1), y)
#     y_pred = model_check_1.predict(X_valid.drop(columns=param, axis=1))
#     dict_exc_param[param] = f1_score(y_valid, y_pred).T

# # Напечатаем трое наилучших f1 с соответствующими исключенными параметрами:
# sorted(dict_exc_param.items(), key=lambda x: -x[1])[:7]

### Проверим выбранную модель с тремя разными вариантами разбиения обучающей выборки:

### 1. LogisticRegression + TrainTestSplit

In [ ]:
model_LR = model
model_LR.fit(X, y)
y_pred_proba = model_LR.predict_proba(X_valid)

matrix = pd.DataFrame()
res = pd.DataFrame(y_pred_proba[:, 1], columns=['prob'])

for threshold in range(50, 65, 1):  # 50, 70, 1
    y_pred_cut = res.prob.apply(lambda x: 1 if x >= threshold/100 else 0)
    T_def_0, F_def_0, F_def_1, T_def_1 = confusion_matrix(y_valid, y_pred_cut).ravel()
    f1 = f1_score(y_valid, y_pred_cut.values).T
    BAS = balanced_accuracy_score(y_valid, y_pred_cut.values)
    matrix = matrix.append(pd.DataFrame(data=[[T_def_0, F_def_0, F_def_1, T_def_1, f1, BAS]], 
                                        columns=['T_def_0', 'F_def_0', 'F_def_1', 'T_def_1', 'f1_score', 'BAS'], 
                                        index=[threshold/100]))

matrix['profit'] = matrix.T_def_0 - matrix.F_def_0 + matrix.T_def_1 - 8 * matrix.F_def_1

# Три лучших результата (по метрике f1) будем записывать в новую таблицу:
top3_model_LR = matrix.sort_values(by='f1_score', ascending=False).iloc[:3]

matrix

In [ ]:
max_f1 = matrix[matrix.f1_score == matrix.f1_score.max()].index
y_pred_prob = pd.DataFrame(model_LR.predict_proba(X_valid))
y_pred = y_pred_prob.iloc[:, -1].apply(lambda x: 1 if x >= max_f1 else 0)
print(classification_report(y_valid, y_pred))

### Построим ROC_AUC:

In [ ]:
y_pred_proba = model_LR.predict_proba(X_valid)
roc_auc = roc_auc_score(y_valid, y_pred_proba[:, 1])
fpr, tpr, thresholds = roc_curve(y_valid, y_pred_proba[:, 1])
plt.figure()
plt.plot([0, 1], label='Baseline', linestyle='--')
plt.plot(fpr, tpr, label='Regression')
plt.title('Logistic Regression ROC AUC = %0.3f' % roc_auc)
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.legend(loc='lower right')

### Проверка модели на переобучение:

In [ ]:
# Чтобы проверить наличие переобученния модели воспользуемся функцией cross_validate:
cv_metrics = cross_validate(model_LR, X, y, cv=5, scoring='f1_micro', return_train_score=True)

def plot_cv_metrics(cv_metrics):
    avg_f1_train, std_f1_train = cv_metrics['train_score'].mean(), cv_metrics['train_score'].std()
    avg_f1_valid, std_f1_valid = cv_metrics['test_score'].mean(), cv_metrics['test_score'].std()
    print('[train] F1-score = {:.2f} +/- {:.2f}'.format(avg_f1_train, std_f1_train))
    print('[valid] F1-score = {:.2f} +/- {:.2f}'.format(avg_f1_valid, std_f1_valid))
    
    plt.figure(figsize=(15, 5))

    plt.plot(cv_metrics['train_score'], label='train', marker='.')
    plt.plot(cv_metrics['test_score'], label='valid', marker='.')

    plt.ylim([0., 1.]);
    plt.xlabel('CV iteration', fontsize=15)
    plt.ylabel('F1-score', fontsize=15)
    plt.legend(fontsize=15)
    
plot_cv_metrics(cv_metrics)

### 2. LogisticRegression + StratifiedShuffleSplit

In [ ]:
SPLIT = 10   # используем для StratifiedShuffleSplit и KFold

In [ ]:
model_LR_sss = model
sss = StratifiedShuffleSplit(n_splits=SPLIT, random_state=SEED)

train_index, valid_index = [split for split in sss.split(X, y)][0]

X_train_sss = X.iloc[train_index, :]
y_train_sss = y[train_index]
X_valid_sss = X.iloc[valid_index, :]
y_valid_sss = y[valid_index]

model_LR_sss.fit(X_train_sss, y_train_sss)
y_pred_proba = model_LR_sss.predict_proba(X_valid)

matrix = pd.DataFrame()
res = pd.DataFrame(y_pred_proba[:, 1], columns=['prob'])

for threshold in range(50, 65, 1):
    y_pred_cut = res.prob.apply(lambda x: 1 if x >= threshold/100 else 0)
    T_def_0, F_def_0, F_def_1, T_def_1 = confusion_matrix(y_valid, y_pred_cut).ravel()
    f1 = f1_score(y_valid, y_pred_cut.values).T
    BAS = balanced_accuracy_score(y_valid, y_pred_cut.values)
    matrix = matrix.append(pd.DataFrame(data=[[T_def_0, F_def_0, F_def_1, T_def_1, f1, BAS]], 
                                        columns=['T_def_0', 'F_def_0', 'F_def_1', 'T_def_1', 'f1_score', 'BAS'], 
                                        index=[threshold/100]))

matrix['profit'] = matrix.T_def_0 - matrix.F_def_0 + matrix.T_def_1 - 8 * matrix.F_def_1

# Будем записывать три лучших результата (по метрике f1) в новую таблицу:
top3_model_LR_sss = matrix.sort_values(by='f1_score', ascending=False).iloc[:3]
matrix

In [ ]:
max_f1 = matrix[matrix.f1_score == matrix.f1_score.max()].index
y_pred_prob = pd.DataFrame(model_LR_sss.predict_proba(X_valid))
y_pred = y_pred_prob.iloc[:, -1].apply(lambda x: 1 if x >= max_f1 else 0)
print(classification_report(y_valid, y_pred))

### 3. LogisticRegression + KFold

In [ ]:
model_LR_kf = model
kf = KFold(n_splits=SPLIT, shuffle=True, random_state=SEED)

for train_index, test_index in kf.split(X, y):
    X_train_kf = X.iloc[train_index, :]
    y_train_kf = y[train_index]
    X_test_kf = X.iloc[test_index, :]
    y_test_kf = y[test_index]
    model_LR_kf.fit(X_train_kf, y_train_kf)

y_pred_proba = model_LR_kf.predict_proba(X_valid)

matrix = pd.DataFrame()
res = pd.DataFrame(y_pred_proba[:, 1], columns=['prob'])

for threshold in range(50, 65, 1):
    y_pred_cut = res.prob.apply(lambda x: 1 if x >= threshold/100 else 0)
    T_def_0, F_def_0, F_def_1, T_def_1 = confusion_matrix(y_valid, y_pred_cut).ravel()
    f1 = f1_score(y_valid, y_pred_cut.values).T
    BAS = balanced_accuracy_score(y_valid, y_pred_cut.values)
    matrix = matrix.append(pd.DataFrame(data=[[T_def_0, F_def_0, F_def_1, T_def_1, f1, BAS]], 
                                        columns=['T_def_0', 'F_def_0', 'F_def_1', 'T_def_1', 'f1_score', 'BAS'], 
                                        index=[threshold/100]))

matrix['profit'] = matrix.T_def_0 - matrix.F_def_0 + matrix.T_def_1 - 8 * matrix.F_def_1

# Будем записывать три лучших результата (по метрике f1) в новую таблицу:
top3_model_LR_kf = matrix.sort_values(by='f1_score', ascending=False).iloc[:3]
matrix

In [ ]:
max_f1 = matrix[matrix.f1_score == matrix.f1_score.max()].index
y_pred_prob = pd.DataFrame(model_LR_kf.predict_proba(X_valid))
y_pred = y_pred_prob.iloc[:, -1].apply(lambda x: 1 if x >= max_f1 else 0)
print(classification_report(y_valid, y_pred))

## 9. Submission

In [ ]:
# Выбирем лучшую модель и порог отсечения:
top_score_df = pd.DataFrame()
top_score_df = pd.concat([top3_model_LR, top3_model_LR_sss, top3_model_LR_kf], keys=['model_LR', 'model_LR_sss', 'model_LR_kf'])
top_score_df

In [ ]:
# Лучшие полученные результаты по f1 на валидации (но с ухудшением на test):
# 0.362998 - model_LR
# 0.364388 - model_LR_sss
# 0.363171 - model_LR_kf

In [ ]:
# На основании рассчитаных ранее результатов по f1 выберем соответвующую модель и порог отсечения:
threshold_cut = 0.588 # загрузив несколько раз submission установим более точно порог отсечения.
predict_submission = model_LR.predict_proba(test)    # X_Over, 0.588; f1=0.35401
# predict_submission = model_LR_sss.predict_proba(test)  
# predict_submission = model_LR_kf.predict_proba(test)

# Загружаем итоговый расчет в 'submission.csv':
sample_submission = pd.read_csv(DATA_DIR+'/sample_submission.csv')
sample_submission['default'] = predict_submission[:, -1]
sample_submission['default'] = sample_submission['default'].apply(lambda x: 1 if x >= threshold_cut else 0)
sample_submission.to_csv('submission.csv', index=False)

print(sample_submission.shape)
sample_submission.head(5)

## 10. Выводы:

Достич максимального результата в 0,35554 (submission на test) фактически удалось за счет следующих действий:
* обработка первичных признаков (группировок, преобразования категориальных в ординальные или числовые признаки), создания новых признаков ('month', 'sex_car') и удаления не эффективных ('client_id', 'car', 'sex');
* нормализации всей выборки (RobustScaler) и балансировки обучающей выборки (RandomOverSampler);
* применения модели LogisticRegression и подбор гиперпараметров методом GridSearchCV;
* определением оптимального порого отсечения вероятности дефолта.

При этом было затрачено огромное кол-во времени в попытках улучшить качество прогноза модели (наверное около 90% от общего затраченного времени), но практически с нулевым результатом (или на уровне прироста +0,001-0,002). Проблема заключалось в том что при определенных изменениях в обработке данных получал прирост f1 для валидационной выборки (до 0,364), но при этом одновременно получал ухудшение результата на итоговом наборе данных (test), что требовало фактически проверки каждого незначительного изменения на submission. Так же усложняло задачу то что для валидационной выборки оптимальный порог отсечения определялся на уровне 0,56-0,57, в то время как для итогового набора данных оптимальный порог отсечения находился в районе 0,59.